# Настраиваем окружение


*   устанавливаем pyspark



In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ad7314ca9b402b3d3d1864dc5af45e3a543f269ed7b85ce7640ef5cbe6837f4f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


# Создаём сессию - точкy входа для spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[4]")\
        .appName("Colab_pyspark")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
sc

<SparkContext master=local[4] appName=Colab_pyspark>

In [5]:
spark

In [6]:
# Просмотр текущей конфигурации
from pyspark import SparkConf

SparkConf().getAll()

[('spark.ui.port', '4050'),
 ('spark.master', 'local[4]'),
 ('spark.app.name', 'Colab_pyspark'),
 ('spark.app.submitTime', '1699277029027'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

# Примеры

Теперь можно попробовать что-то запустить. Весь код ниже мы ещё обсудим на уроке, пока что он просто для примера (если что-то не понятно, не заморачивайтесь).

## Создаём dataframe и выводим его

In [7]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname", "middlename", "lastname", "dob", "gender", "salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [8]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



## Считываем и сохраняем файл

Загружаем файл, который будем считывать

In [9]:
!wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv

--2023-11-06 13:25:26--  https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5329268 (5.1M) [text/plain]
Saving to: ‘PD-data-train.csv’

PD-data-train.csv   100%[===================>]   5.08M  --.-KB/s    in 0.06s   

2023-11-06 13:25:26 (80.6 MB/s) - ‘PD-data-train.csv’ saved [5329268/5329268]



In [10]:
df = spark.read.csv("PD-data-train.csv", header=True, inferSchema=True, sep=';')

In [11]:
df.show()

+---------+----------+-----------------+------------+-------------------+----------------------+--------------+----------------------------+--------------------+--------+--------------------------+-------------+---------------------+------------------------+------------+----------------------+-----------------------+----------------------+---------+--------------+-------------------+------------------------+-----------------------+-------------------+-------------------+-------+--------+--------------+---------------+--------------+--------------+--------------+---------------+-------------+----------------+--------------------+-----------+
|record_id|ar_revenue|ar_total_expenses|ar_sale_cost|ar_selling_expenses|ar_management_expenses|ar_sale_profit|ar_balance_of_rvns_and_expns|ar_profit_before_tax|ar_taxes|ar_other_profit_and_losses|ar_net_profit|ab_immobilized_assets|ab_mobile_current_assets|ab_inventory|ab_accounts_receivable|ab_other_current_assets|ab_cash_and_securities|ab_losses|

In [12]:
df.write.parquet('PD-data-train.parquet')

In [13]:
df.write.parquet('PD-data-train.snappy.parquet', compression='snappy')

In [14]:
!ls -l -h

total 5.1M
-rw-r--r-- 1 root root 5.1M Nov  6 13:25 PD-data-train.csv
drwxr-xr-x 2 root root 4.0K Nov  6 13:25 PD-data-train.parquet
drwxr-xr-x 2 root root 4.0K Nov  6 13:26 PD-data-train.snappy.parquet
drwxr-xr-x 1 root root 4.0K Nov  3 18:00 sample_data


In [15]:
!pwd

/content
